In [1]:
# pip install langchain-google-genai 
# pip install langchain

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
google_gemini_api=os.getenv("GOOGLE_API_KEY")

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm_model=ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=google_gemini_api)

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1721506661.682962  298750 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1721506661.689778  298750 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721506661.690683  298750 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [5]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [6]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for the disease {disease_name} for public in {tone} tone for the awareness. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [7]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "disease_name", "tone", "response_json"],
    template=TEMPLATE
)

In [8]:
quiz_chain=LLMChain(llm=llm_model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

/Users/anuragsingh/Documents/GitHub/Mr-analyzer/analyzerenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
TEMPLATE2="""
You are an medical expert and health advocates or public health advocates. Given a Multiple Choice Quiz for {disease_name} disease.\
You need to evaluate the quality of the question and give a complete analysis of the quiz. Only use at max 50 words for quality analysis. 
if the quiz is not at per with the quality and awreness and helpfull related to the public,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the public awareness and health
Quiz_MCQs:
{quiz}

Check from an medical expert and health advocates or public health advocates of the above quiz:
"""

In [10]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["disease_name", "quiz"], template=TEMPLATE2)

In [11]:
review_chain=LLMChain(llm=llm_model, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [12]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "disease_name", "tone", "response_json"],
    output_variables=["quiz", "review"], 
    verbose=True,
)

## loading the data for generating the MCQs

In [15]:
import pandas as pd
import json

In [16]:
new_df=pd.read_csv("data.csv")

In [17]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
response=generate_evaluate_chain(
    {
        "text":new_df.text,
        "number":new_df.number,
        "disease_name":new_df.disease_name,
        "tone":new_df.tone,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:0    Jaundice is a medical condition characterized ...
Name: text, dtype: object
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of 0    5
Name: number, dtype: int64 multiple choice questions for the disease 0    Jaundice
Name: disease_name, dtype: object for public in 0    good
Name: tone, dtype: object tone for the awareness. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make 0    5
Name: number, dtype: int64 MCQs
### RESPONSE_JSON
{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "ch

In [21]:
response

{'text': 0    Jaundice is a medical condition characterized ...
 Name: text, dtype: object,
 'number': 0    5
 Name: number, dtype: int64,
 'disease_name': 0    Jaundice
 Name: disease_name, dtype: object,
 'tone': 0    good
 Name: tone, dtype: object,
 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}',
 'quiz': '### RESPONSE_JSON\n{"1": {"mcq": "What is the medical condition characterized by yellowing of the skin and whites of the eyes?", "options": {"a": "Jaundice", "b": "Cirrhosis", "c": "Hepatitis", "d": "Hemochromatosis"}, "cor

In [37]:
new_quiz_df=pd.DataFrame([response])

In [43]:
quiz_json = new_quiz_df.loc[0, 'quiz'].split('### RESPONSE_JSON\n')[1]
quiz_data = json.loads(quiz_json)

# Display the quiz questions
for key, value in quiz_data.items():
    print(f"Question {key}: {value['mcq']}")
    for option_key, option_value in value['options'].items():
        print(f"  {option_key}: {option_value}")
    print(f"Correct Answer: {value['correct']}\n")

Question 1: What is the medical condition characterized by yellowing of the skin and whites of the eyes?
  a: Jaundice
  b: Cirrhosis
  c: Hepatitis
  d: Hemochromatosis
Correct Answer: a

Question 2: Which of the following is a symptom of jaundice?
  a: Yellowing of the skin
  b: Dark urine
  c: Pale stools
  d: All of the above
Correct Answer: d

Question 3: What is the most common cause of jaundice in newborns?
  a: Breast milk jaundice
  b: Liver disease
  c: Gallstones
  d: Pancreatitis
Correct Answer: a

Question 4: Which of the following tests is used to diagnose jaundice?
  a: Blood test
  b: Liver biopsy
  c: Ultrasound
  d: All of the above
Correct Answer: d

Question 5: What is the treatment for jaundice?
  a: Medication
  b: Surgery
  c: Lifestyle changes
  d: A combination of treatments
Correct Answer: d



In [44]:
quiz_list = []
for key, value in quiz_data.items():
    question = value['mcq']
    options = value['options']
    correct_ans = value['correct']
    row = {
        'question': question,
        'optiona': options.get('a', ''),
        'optionb': options.get('b', ''),
        'optionc': options.get('c', ''),
        'optiond': options.get('d', ''),
        'correctans': correct_ans
    }
    quiz_list.append(row)

# Convert to DataFrame
df_quiz = pd.DataFrame(quiz_list)

# Save to CSV
df_quiz.to_csv('quiz_questions.csv', index=False)